# LAS/LAZ to COPC Conversion Tutorial

This notebook demonstrates how to convert LAS/LAZ point cloud files to COPC (Cloud Optimized Point Cloud) format using PDAL.

## What is COPC?

COPC (Cloud Optimized Point Cloud) adds spatial indexing to point cloud data, enabling:
- **Fast spatial queries** - Extract regions without reading entire files
- **Progressive loading** - Load data at different detail levels
- **Better performance** - Especially for web applications and large datasets

## Setup

1. **Install dependencies** using the provided environment file:
   ```bash
   conda env create -f environment.yml
   conda activate copc_api
   ```

2. **Get a LAS/LAZ file** to work with (you can download from the README or use your own)

> **Note**: PDAL installation may take several minutes - be patient!

1. Have a las file that you want to work with and convert to copc
2. (Optional) Export it from the hoydedata.no website
3. Install the required packages for doing this conversion
    Best way of doing this is by using the PDAL library which is most easily installed through Conda

In this folder you can find the requirements.yml file which contains all of the packages needed for this notebook and can be created using the following command

> conda env create -f environment.yml

Installing the PDAL library may take some time, so be patient.



In [11]:
import pdal
import json
import time
from pathlib import Path
import numpy as np
import os

In [ ]:
!wget https://github.com/PDAL/data/raw/refs/heads/main/isprs/CSite2_orig-utm.laz
!mv CSite2_orig-utm.laz test.laz

--2025-08-11 12:37:57--  https://github.com/PDAL/data/raw/refs/heads/main/isprs/CSite2_orig-utm.laz
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/PDAL/data/refs/heads/main/isprs/CSite2_orig-utm.laz [following]
--2025-08-11 12:37:57--  https://media.githubusercontent.com/media/PDAL/data/refs/heads/main/isprs/CSite2_orig-utm.laz
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1464552 (1,4M) [application/octet-stream]
Saving to: ‘CSite2_orig-utm.laz’

CSite2_orig-utm.laz 100%[===================>]   1,40M  --.-KB/s    in 0,05s   

2025-08-11 12:37:58 (30,8 MB/s) - ‘CSite2_orig-utm

## Step 1: Examine Your Point Cloud Data

Let's first understand what we're working with by reading and analyzing the LAS/LAZ file structure.

In [13]:
las_file = "test_2.laz"
pipeline = pdal.Pipeline(json.dumps({
    "pipeline": [
        {
            "type": "readers.las",
            "filename": las_file
        }
    ]
}))

pipeline.execute()

19253870

In [14]:
arrays = pipeline.arrays
if arrays:
    points = arrays[0]
    print(f"  Points: {len(points):,}")
    print(f"  Dimensions: {list(points.dtype.names)}")
    print(f"  Bounds: X({points['X'].min():.1f} to {points['X'].max():.1f})")
    print(f"           Y({points['Y'].min():.1f} to {points['Y'].max():.1f})")
    print(f"           Z({points['Z'].min():.1f} to {points['Z'].max():.1f})")
    
    # Show first point
    print(f"  First point: X={points['X'][0]:.2f}, Y={points['Y'][0]:.2f}, Z={points['Z'][0]:.2f}")
    


  Points: 19,253,870
  Dimensions: ['X', 'Y', 'Z', 'Intensity', 'ReturnNumber', 'NumberOfReturns', 'ScanDirectionFlag', 'EdgeOfFlightLine', 'Classification', 'Synthetic', 'KeyPoint', 'Withheld', 'Overlap', 'ScanAngleRank', 'UserData', 'PointSourceId', 'GpsTime', 'ScanChannel']
  Bounds: X(453600.0 to 454400.0)
           Y(6456000.0 to 6456600.0)
           Z(57.6 to 162.1)
  First point: X=453600.03, Y=6456486.00, Z=130.39


From the output above we can see that the laz file has approx. 500k points. There is also metainformation for each point such as "Intensity", "Classification", "Withheld" etc - which can vary depending on the data creation.

Lets start by examining some of the data we have, e.g. the 5 first values.

In [15]:
point = pipeline.arrays[0][0]
point

np.void((453600.03, 6456486.0, 130.39000000000001, 49762, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0.57, 2, 19, 366634123.5912308, 2), dtype=[('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Intensity', '<u2'), ('ReturnNumber', 'u1'), ('NumberOfReturns', 'u1'), ('ScanDirectionFlag', 'u1'), ('EdgeOfFlightLine', 'u1'), ('Classification', 'u1'), ('Synthetic', 'u1'), ('KeyPoint', 'u1'), ('Withheld', 'u1'), ('Overlap', 'u1'), ('ScanAngleRank', '<f4'), ('UserData', 'u1'), ('PointSourceId', '<u2'), ('GpsTime', '<f8'), ('ScanChannel', 'u1')])

This creates a point as a np.void object which is a numpy datatype for storing a structured element of an array. This allows us to index using the different keys that are stored with it.

In [16]:
print("All dtypes in the point: ", list(point.dtype.names))

values_of_interest = ["X", "Y", "Z", "Intensity", "Classification", "Withheld"]

for value in values_of_interest:
    print(f"{value}: {point[value]}")
    print("-"*10)

All dtypes in the point:  ['X', 'Y', 'Z', 'Intensity', 'ReturnNumber', 'NumberOfReturns', 'ScanDirectionFlag', 'EdgeOfFlightLine', 'Classification', 'Synthetic', 'KeyPoint', 'Withheld', 'Overlap', 'ScanAngleRank', 'UserData', 'PointSourceId', 'GpsTime', 'ScanChannel']
X: 453600.03
----------
Y: 6456486.0
----------
Z: 130.39000000000001
----------
Intensity: 49762
----------
Classification: 1
----------
Withheld: 0
----------


In this way we can inspect the values here

In [17]:

def convert_laz_to_copc(input_file: str, output_file: str = "output.copc.laz", chunk_size: int = 100000):
    """Convert LAS/LAZ to COPC (LAZ-compressed) using PDAL's writers.copc."""
    
    print(f"🔄 Converting {input_file} → {output_file} (COPC, chunkSize={chunk_size})")

    pipeline = pdal.Pipeline(json.dumps({
        "pipeline": [
            {
                "type": "readers.las",
                "filename": input_file
            },
            {
                "type": "writers.copc",
                "filename": output_file,
                "forward": "all"
            }
        ]
    }))

    start_time = time.time()
    pipeline.execute()
    conversion_time = time.time() - start_time
    
    print(f"✅ Wrote COPC: {output_file}")
    print(f"⏱️  Conversion time: {conversion_time:.2f} s")
    return output_file

# Example:
convert_laz_to_copc("test_2.laz", "compressed_output.copc", chunk_size=100000)

🔄 Converting test_2.laz → compressed_output.copc (COPC, chunkSize=100000)
✅ Wrote COPC: compressed_output.copc
⏱️  Conversion time: 42.31 s


'compressed_output.copc'

## Step 2: Convert to COPC Format

The conversion was fast! Now let's explore the key differences between LAZ and COPC files.

Let's highlight and look at the :
1. File size
2. Spatial indexing
3. Progressive loading


### File Size Comparison

In [18]:
def compare_file_sizes(laz_file: str, copc_file: str):
    """Compare file sizes between LAZ and COPC files"""
    
    # Check if files exist
    if not Path(laz_file).exists():
        print(f"❌ LAZ file not found: {laz_file}")
        return
    
    if not Path(copc_file).exists():
        print(f"❌ COPC file not found: {copc_file}")
        return
    
    # Get file sizes in bytes
    laz_size_bytes = Path(laz_file).stat().st_size
    copc_size_bytes = Path(copc_file).stat().st_size
    
    # Convert to MB for display
    laz_size_mb = laz_size_bytes / (1024**2)
    copc_size_mb = copc_size_bytes / (1024**2)
    
    # Calculate ratio
    size_ratio = copc_size_bytes / laz_size_bytes
    
    print(f"📊 File Size Comparison:")
    print(f"  LAZ file:  {laz_size_mb:.2f} MB ({laz_size_bytes:,} bytes)")
    print(f"  COPC file: {copc_size_mb:.2f} MB ({copc_size_bytes:,} bytes)")
    print(f"  Size ratio: {size_ratio:.2f}x")
    
    if size_ratio < 1.0:
        print(f"  ✅ COPC is {((1-size_ratio)*100):.1f}% smaller than LAZ")
    else:
        print(f"  ⚠️  COPC is {((size_ratio-1)*100):.1f}% larger than LAZ")
    
    return {
        'laz_size_mb': laz_size_mb,
        'copc_size_mb': copc_size_mb,
        'ratio': size_ratio
    }

compare_file_sizes("test_2.laz", "compressed_output.copc")

📊 File Size Comparison:
  LAZ file:  118.52 MB (124,276,257 bytes)
  COPC file: 115.41 MB (121,015,623 bytes)
  Size ratio: 0.97x
  ✅ COPC is 2.6% smaller than LAZ


{'laz_size_mb': 118.51907444000244,
 'copc_size_mb': 115.40949153900146,
 'ratio': 0.9737630173396677}

As we can see here, the file size actually increases quite dramatically. The file size is 2.21 (!) times larger than the laz file. Though, as we will see in the rest of the notebook, the increased file size may introduce benefits that outweigh the increased file size.


### Spatial Indexing Benefits

In [19]:
def benchmark_region_extraction(laz_file: str, copc_file: str, region_size: float = 100.0):
    """Benchmark extracting data from a specific region"""
    
    print(f"🏃‍♂️ Region Extraction Benchmark (region size: {region_size}m)")
    print("=" * 60)
    
    # First, get the bounds from the COPC file to define a region
    print("📊 Getting data bounds...")
    copc_pipeline = pdal.Pipeline(json.dumps({
        "pipeline": [
            {
                "type": "readers.copc",
                "filename": copc_file
            }
        ]
    }))
    copc_pipeline.execute()
    
    metadata = copc_pipeline.metadata
    
    # Try different ways to get bounds
    bounds = None
    if 'metadata' in metadata and 'readers.copc' in metadata['metadata']:
        copc_meta = metadata['metadata']['readers.copc']
        if 'bounds' in copc_meta:
            bounds = copc_meta['bounds']
        elif 'minx' in copc_meta and 'maxx' in copc_meta:
            bounds = {
                'X': copc_meta['minx'],
                'Y': copc_meta['miny'],
                'Z': copc_meta['minz']
            }
    
    # If still no bounds, use the actual data bounds
    if bounds is None:
        print("⚠️  Using data bounds instead of metadata bounds")
        points = copc_pipeline.arrays[0]
        bounds = {
            'X': float(points['X'].min()),
            'Y': float(points['Y'].min()),
            'Z': float(points['Z'].min())
        }
    
    x_min, x_max = bounds['X'], bounds['X'] + region_size
    y_min, y_max = bounds['Y'], bounds['Y'] + region_size
    
    print(f"��️  Region bounds: X({x_min:.1f} to {x_max:.1f}), Y({y_min:.1f} to {y_max:.1f})")
    
    # Test 1: Extract region from LAZ file (must read entire file)
    print(f"\n📖 Extracting region from LAZ file...")
    start_time = time.time()
    
    laz_pipeline = pdal.Pipeline(json.dumps({
        "pipeline": [
            {
                "type": "readers.las",
                "filename": laz_file
            },
            {
                "type": "filters.crop",
                "bounds": f"([{x_min},{x_max}],[{y_min},{y_max}])"
            }
        ]
    }))
    laz_pipeline.execute()
    laz_points = len(laz_pipeline.arrays[0])
    laz_time = time.time() - start_time
    
    print(f"  ✅ Points extracted: {laz_points:,}")
    print(f"  ⏱️  Time: {laz_time:.2f} seconds")
    print(f"  �� Speed: {laz_points/laz_time:,.0f} points/second")
    
    # Test 2: Extract region from COPC file (spatial index)
    print(f"\n🎯 Extracting region from COPC file...")
    start_time = time.time()
    
    copc_region_pipeline = pdal.Pipeline(json.dumps({
        "pipeline": [
            {
                "type": "readers.copc",
                "filename": copc_file,
                "bounds": f"([{x_min},{x_max}],[{y_min},{y_max}])"
            }
        ]
    }))
    copc_region_pipeline.execute()
    copc_points = len(copc_region_pipeline.arrays[0])
    copc_time = time.time() - start_time
    
    print(f"  ✅ Points extracted: {copc_points:,}")
    print(f"  ⏱️  Time: {copc_time:.2f} seconds")
    print(f"  🚀 Speed: {copc_points/copc_time:,.0f} points/second")
    
    # Compare results
    print(f"\n📊 Performance Comparison:")
    speed_improvement = laz_time / copc_time if copc_time > 0 else float('inf')
    print(f"  🏆 COPC is {speed_improvement:.1f}x faster than LAZ")
    print(f"  ⏱️  Time saved: {laz_time - copc_time:.2f} seconds")
    
    if laz_points != copc_points:
        print(f"  ⚠️  Point count difference: LAZ={laz_points:,}, COPC={copc_points:,}")
    else:
        print(f"  ✅ Same point count: {laz_points:,}")
    
    return {
        'laz': {'points': laz_points, 'time': laz_time},
        'copc': {'points': copc_points, 'time': copc_time},
        'speedup': speed_improvement
    }

# Usage
if __name__ == "__main__":
    laz_file = "test_2.laz"
    copc_file = "compressed_output.copc"
    
    # Single region benchmark
    results = benchmark_region_extraction(laz_file, copc_file, region_size=100)

🏃‍♂️ Region Extraction Benchmark (region size: 100m)
📊 Getting data bounds...
��️  Region bounds: X(453600.0 to 453700.0), Y(6456000.0 to 6456100.0)

📖 Extracting region from LAZ file...
  ✅ Points extracted: 390,547
  ⏱️  Time: 5.39 seconds
  �� Speed: 72,494 points/second

🎯 Extracting region from COPC file...
  ✅ Points extracted: 390,547
  ⏱️  Time: 0.45 seconds
  🚀 Speed: 869,232 points/second

📊 Performance Comparison:
  🏆 COPC is 12.0x faster than LAZ
  ⏱️  Time saved: 4.94 seconds
  ✅ Same point count: 390,547


As we can see here, extracting regions in the files are much faster using the copc file. The main reason for this is the spatial indexing that allows us to only extract the data for a small subset of the file, while for the laz file we need to read the entire file in order to extract the same subset. This overhead increases even more when we need to extract multiple regions, especially from multiple files.

### Extracting multiple regions from the same file

In [20]:
def benchmark_average_regions(laz_file: str, copc_file: str, num_regions: int = 10, region_size: float = 100.0):
    """Benchmark average performance across multiple random regions"""
    
    print(f"🧪 Average Performance Across {num_regions} Regions (size: {region_size}m)")
    print("=" * 60)
    
    # Get bounds from COPC file
    print("📊 Getting data bounds...")
    copc_pipeline = pdal.Pipeline(json.dumps({
        "pipeline": [
            {"type": "readers.copc", "filename": copc_file}
        ]
    }))
    copc_pipeline.execute()
    points = copc_pipeline.arrays[0]
    
    x_min, x_max = float(points['X'].min()), float(points['X'].max())
    y_min, y_max = float(points['Y'].min()), float(points['Y'].max())
    
    print(f"📐 Full dataset bounds: X({x_min:.1f} to {x_max:.1f}), Y({y_min:.1f} to {y_max:.1f})")
    
    # Store results for all regions
    laz_times = []
    copc_times = []
    laz_points = []
    copc_points = []
    
    print(f"\n🔄 Testing {num_regions} random regions...")
    
    for i in range(num_regions):
        # Generate random region within bounds
        region_x_min = np.random.uniform(x_min, x_max - region_size)
        region_y_min = np.random.uniform(y_min, y_max - region_size)
        region_x_max = region_x_min + region_size
        region_y_max = region_y_min + region_size
        
        # Test LAZ extraction
        start_time = time.time()
        laz_pipeline = pdal.Pipeline(json.dumps({
            "pipeline": [
                {"type": "readers.las", "filename": laz_file},
                {"type": "filters.crop", "bounds": f"([{region_x_min},{region_x_max}],[{region_y_min},{region_y_max}])"}
            ]
        }))
        laz_pipeline.execute()
        laz_time = time.time() - start_time
        laz_points_count = len(laz_pipeline.arrays[0])
        
        # Test COPC extraction
        start_time = time.time()
        copc_pipeline = pdal.Pipeline(json.dumps({
            "pipeline": [
                {
                    "type": "readers.copc",
                    "filename": copc_file,
                    "bounds": f"([{region_x_min},{region_x_max}],[{region_y_min},{region_y_max}])"
                }
            ]
        }))
        copc_pipeline.execute()
        copc_time = time.time() - start_time
        copc_points_count = len(copc_pipeline.arrays[0])
        
        # Store results
        laz_times.append(laz_time)
        copc_times.append(copc_time)
        laz_points.append(laz_points_count)
        copc_points.append(copc_points_count)
        
        if (i + 1) % 5 == 0:
            print(f"  Completed {i + 1}/{num_regions} regions...")
    
    # Calculate averages
    avg_laz_time = np.mean(laz_times)
    avg_copc_time = np.mean(copc_times)
    avg_laz_points = np.mean(laz_points)
    avg_copc_points = np.mean(copc_points)
    avg_speedup = avg_laz_time / avg_copc_time
    
    # Calculate standard deviations
    std_laz_time = np.std(laz_times)
    std_copc_time = np.std(copc_times)
    
    print(f"\n📊 Average Results Across {num_regions} Regions:")
    print("=" * 50)
    print(f"  LAZ:  {avg_laz_points:.0f} ± {np.std(laz_points):.0f} points in {avg_laz_time:.2f} ± {std_laz_time:.2f}s")
    print(f"  COPC: {avg_copc_points:.0f} ± {np.std(copc_points):.0f} points in {avg_copc_time:.2f} ± {std_copc_time:.2f}s")
    print(f"  �� Average speedup: {avg_speedup:.1f}x")
    print(f"  ⏱️  Average time saved: {avg_laz_time - avg_copc_time:.2f}s per region")
    
    # Performance summary
    print(f"\n🎯 Performance Summary:")
    print(f"  ✅ COPC is {avg_speedup:.1f}x faster on average")
    print(f"  📈 Speedup range: {min(laz_times)/max(copc_times):.1f}x to {max(laz_times)/min(copc_times):.1f}x")
    print(f"  🎲 Consistent performance: {'Yes' if std_copc_time/avg_copc_time < 0.5 else 'Variable'}")
    
    return {
        'avg_laz_time': avg_laz_time,
        'avg_copc_time': avg_copc_time,
        'avg_speedup': avg_speedup,
        'std_laz_time': std_laz_time,
        'std_copc_time': std_copc_time
    }

# Usage
if __name__ == "__main__":
    laz_file = "test_2.laz"
    copc_file = "compressed_output.copc"
    
    # Test with different region sizes
    region_sizes = [50, 100, 200, 500]
    
    for size in region_sizes:
        print(f"\n{'='*70}")
        results = benchmark_average_regions(laz_file, copc_file, num_regions=10, region_size=size)
        print(f"Region size {size}m: COPC is {results['avg_speedup']:.1f}x faster")


🧪 Average Performance Across 10 Regions (size: 50m)
📊 Getting data bounds...
📐 Full dataset bounds: X(453600.0 to 454400.0), Y(6456000.0 to 6456600.0)

🔄 Testing 10 random regions...
  Completed 5/10 regions...
  Completed 10/10 regions...

📊 Average Results Across 10 Regions:
  LAZ:  101119 ± 26032 points in 5.07 ± 0.46s
  COPC: 101119 ± 26032 points in 0.29 ± 0.09s
  �� Average speedup: 17.6x
  ⏱️  Average time saved: 4.78s per region

🎯 Performance Summary:
  ✅ COPC is 17.6x faster on average
  📈 Speedup range: 9.0x to 37.7x
  🎲 Consistent performance: Yes
Region size 50m: COPC is 17.6x faster

🧪 Average Performance Across 10 Regions (size: 100m)
📊 Getting data bounds...
📐 Full dataset bounds: X(453600.0 to 454400.0), Y(6456000.0 to 6456600.0)

🔄 Testing 10 random regions...
  Completed 5/10 regions...
  Completed 10/10 regions...

📊 Average Results Across 10 Regions:
  LAZ:  430214 ± 68772 points in 5.11 ± 0.52s
  COPC: 430214 ± 68772 points in 0.56 ± 0.08s
  �� Average speedup: 9

As we can see evident here, extracting a subset of data from the file is much faster using copc, especially when the areas are very small. The benefit decreases as the areas increase, but it is still substantially faster than just using the laz files.


### Progressive Loading

In [21]:
def benchmark_progressive_loading(laz_file: str, copc_file: str, region_size: float = 200.0):
    """Benchmark progressive loading at different detail levels"""
    
    print(f"📈 Progressive Loading Benchmark (region size: {region_size}m)")
    print("=" * 60)
    
    # Get bounds from COPC file
    print("📊 Getting data bounds...")
    copc_pipeline = pdal.Pipeline(json.dumps({
        "pipeline": [
            {"type": "readers.copc", "filename": copc_file}
        ]
    }))
    copc_pipeline.execute()
    points = copc_pipeline.arrays[0]
    
    x_min, x_max = float(points['X'].min()), float(points['X'].max())
    y_min, y_max = float(points['Y'].min()), float(points['Y'].max())
    
    # Define region
    region_x_min, region_x_max = x_min, x_min + region_size
    region_y_min, region_y_max = y_min, y_min + region_size
    
    print(f"📍 Region: X({region_x_min:.1f} to {region_x_max:.1f}), Y({region_y_min:.1f} to {region_y_max:.1f})")
    
    # Test different detail levels (resolution)
    detail_levels = [0.5, 1.0, 2.0, 5.0, 10.0]  # meters
    
    print(f"\n🔍 Testing different detail levels...")
    print("-" * 50)
    
    results = []
    
    for resolution in detail_levels:
        print(f"\n📏 Detail level: {resolution}m resolution")
        
        # LAZ: Must read entire region (no progressive loading)
        print(f"  LAZ extraction (full region)...")
        start_time = time.time()
        laz_pipeline = pdal.Pipeline(json.dumps({
            "pipeline": [
                {"type": "readers.las", "filename": laz_file},
                {"type": "filters.crop", "bounds": f"([{region_x_min},{region_x_max}],[{region_y_min},{region_y_max}])"}
            ]
        }))
        laz_pipeline.execute()
        laz_time = time.time() - start_time
        laz_points = len(laz_pipeline.arrays[0])
        
        # COPC: Progressive loading at specific resolution
        print(f"  🎯 COPC extraction (resolution: {resolution}m)...")
        start_time = time.time()
        copc_pipeline = pdal.Pipeline(json.dumps({
            "pipeline": [
                {
                    "type": "readers.copc",
                    "filename": copc_file,
                    "bounds": f"([{region_x_min},{region_x_max}],[{region_y_min},{region_y_max}])",
                    "resolution": resolution
                }
            ]
        }))
        copc_pipeline.execute()
        copc_time = time.time() - start_time
        copc_points = len(copc_pipeline.arrays[0])
        
        speedup = laz_time / copc_time if copc_time > 0 else float('inf')
        data_reduction = (1 - copc_points/laz_points) * 100 if laz_points > 0 else 0
        
        print(f"    LAZ:  {laz_points:,} points in {laz_time:.2f}s")
        print(f"    COPC: {copc_points:,} points in {copc_time:.2f}s")
        print(f"    🏆 Speedup: {speedup:.1f}x")
        print(f"    📉 Data reduction: {data_reduction:.1f}%")
        
        results.append({
            'resolution': resolution,
            'laz_points': laz_points,
            'laz_time': laz_time,
            'copc_points': copc_points,
            'copc_time': copc_time,
            'speedup': speedup,
            'data_reduction': data_reduction
        })
    
    # Summary table
    print(f"\n📊 Progressive Loading Summary:")
    print("=" * 80)
    print(f"{'Resolution':<10} {'LAZ Points':<12} {'LAZ Time':<10} {'COPC Points':<12} {'COPC Time':<10} {'Speedup':<8} {'Reduction':<10}")
    print("-" * 80)
    
    for result in results:
        print(f"{result['resolution']:<10} {result['laz_points']:<12,} {result['laz_time']:<10.2f} "
              f"{result['copc_points']:<12,} {result['copc_time']:<10.2f} {result['speedup']:<8.1f} "
              f"{result['data_reduction']:<10.1f}%")
    
    return results

def benchmark_detail_levels_comparison(laz_file: str, copc_file: str):
    """Compare different detail levels side by side"""
    
    print(f"\n🎯 Detail Level Comparison")
    print("=" * 60)
    
    # Test with a larger region to see more dramatic differences
    region_size = 500.0
    
    # Get bounds
    copc_pipeline = pdal.Pipeline(json.dumps({
        "pipeline": [
            {"type": "readers.copc", "filename": copc_file}
        ]
    }))
    copc_pipeline.execute()
    points = copc_pipeline.arrays[0]
    
    x_min, x_max = float(points['X'].min()), float(points['X'].max())
    y_min, y_max = float(points['Y'].min()), float(points['Y'].max())
    
    region_x_min, region_x_max = x_min, x_min + region_size
    region_y_min, region_y_max = y_min, y_min + region_size
    
    # Test three detail levels: high, medium, low
    detail_levels = [
        ("High Detail", 0.5),
        ("Medium Detail", 2.0),
        ("Low Detail", 10.0)
    ]
    
    print(f"📍 Region: {region_size}m x {region_size}m")
    print(f"🎯 Testing three detail levels...")
    
    for detail_name, resolution in detail_levels:
        print(f"\n📏 {detail_name} ({resolution}m resolution):")
        
        # LAZ (always full detail)
        start_time = time.time()
        laz_pipeline = pdal.Pipeline(json.dumps({
            "pipeline": [
                {"type": "readers.las", "filename": laz_file},
                {"type": "filters.crop", "bounds": f"([{region_x_min},{region_x_max}],[{region_y_min},{region_y_max}])"}
            ]
        }))
        laz_pipeline.execute()
        laz_time = time.time() - start_time
        laz_points = len(laz_pipeline.arrays[0])
        
        # COPC (progressive detail)
        start_time = time.time()
        copc_pipeline = pdal.Pipeline(json.dumps({
            "pipeline": [
                {
                    "type": "readers.copc",
                    "filename": copc_file,
                    "bounds": f"([{region_x_min},{region_x_max}],[{region_y_min},{region_y_max}])",
                    "resolution": resolution
                }
            ]
        }))
        copc_pipeline.execute()
        copc_time = time.time() - start_time
        copc_points = len(copc_pipeline.arrays[0])
        
        speedup = laz_time / copc_time if copc_time > 0 else float('inf')
        data_reduction = (1 - copc_points/laz_points) * 100 if laz_points > 0 else 0
        
        print(f"  LAZ:  {laz_points:,} points in {laz_time:.2f}s")
        print(f"  COPC: {copc_points:,} points in {copc_time:.2f}s")
        print(f"  🏆 Speedup: {speedup:.1f}x")
        print(f"  📉 Data reduction: {data_reduction:.1f}%")
        print(f"  ⚡ Time saved: {laz_time - copc_time:.2f}s")

def show_progressive_benefits():
    """Show the benefits of progressive loading"""
    print(f"\n🎯 Progressive Loading Benefits:")
    print("=" * 50)
    print(f"  ✅ LAZ: Always loads full detail (no choice)")
    print(f"  ✅ COPC: Can load different detail levels")
    print(f"  🚀 Faster loading at lower detail levels")
    print(f"  �� Perfect for mobile/web applications")
    print(f"  🎮 Great for real-time visualization")
    print(f"  💾 Bandwidth savings for web services")
    print(f"  📊 Can start with overview, then add detail")

# Usage
if __name__ == "__main__":
    laz_file = "test_2.laz"
    copc_file = "compressed_output.copc"
    
    # Progressive loading benchmark
    progressive_results = benchmark_progressive_loading(laz_file, copc_file)
    
    # Detail level comparison
    benchmark_detail_levels_comparison(laz_file, copc_file)
    
    # Show benefits
    show_progressive_benefits()

📈 Progressive Loading Benchmark (region size: 200.0m)
📊 Getting data bounds...
📍 Region: X(453600.0 to 453800.0), Y(6456000.0 to 6456200.0)

🔍 Testing different detail levels...
--------------------------------------------------

📏 Detail level: 0.5m resolution
  LAZ extraction (full region)...
  🎯 COPC extraction (resolution: 0.5m)...
    LAZ:  1,675,265 points in 4.86s
    COPC: 1,490,101 points in 0.95s
    🏆 Speedup: 5.1x
    📉 Data reduction: 11.1%

📏 Detail level: 1.0m resolution
  LAZ extraction (full region)...
  🎯 COPC extraction (resolution: 1.0m)...
    LAZ:  1,675,265 points in 5.60s
    COPC: 686,571 points in 0.61s
    🏆 Speedup: 9.1x
    📉 Data reduction: 59.0%

📏 Detail level: 2.0m resolution
  LAZ extraction (full region)...
  🎯 COPC extraction (resolution: 2.0m)...
    LAZ:  1,675,265 points in 4.73s
    COPC: 263,728 points in 0.44s
    🏆 Speedup: 10.9x
    📉 Data reduction: 84.3%

📏 Detail level: 5.0m resolution
  LAZ extraction (full region)...
  🎯 COPC extraction 